# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Для начала необходимо посмотреть на типы данных и их размер, посмотреть что из себя представляет сама таблица: распечатать ее часть

In [74]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

df.info() 
df.head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Одни и те же численные данные имеют разный числовой формат, хотя выражают один количественный тип: например, children - int64, days_employed - float64. Выражать трудовой стаж заемщика дробным числом - неправильно. Кроме того, нахожу странным обозначать его отрицательным числом. Предполагаю, что численные данные необходимо привести к одному типу int64.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем количество пропусков и посмотрим в каких столбцах они находятся.

In [75]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

Обнаружено 2174 пустых значения в столбце days_employed и total_income. Это 10 процентов от общего числа наблюдений.
Возможные причины появления пропусков в данных: преднамеренный отказ их указывать, либо человек был безработный.
Недостающие значения были заполнены медианными значениями на основаннии данных других заемщиков. Так как медианное значение более точно отражает среднее знаечние в вопросах дохода группы лиц.

In [76]:
df.loc[df['days_employed'].isnull(), 'days_employed'] = df['days_employed'].median()
df.loc[df['total_income'].isnull(), 'total_income'] = df['total_income'].median()
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим некорректные данные в столбцах

In [77]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обнаружены отрицательные знаечние в графе "Дети". Сосчитаем объем отрицательных значений в этой графе.

In [78]:
print(df[df['children'] < 0].count())

children            47
days_employed       47
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        47
purpose             47
dtype: int64


Объем наблюдений с отрицательными значениями в графе "Children" составляет 2%. Истинная причина появления отрицательных значение в этой графе неизвестна. Так как один из выводов исследования напрямую зависит от наличия детей, то каким-либо образом преобразовывать отрицательные значения опасно и может некорректно повлиять на результаты исследования. На мой взгляд, такие строки лучше удалить полностью. 

In [79]:
df.loc[df['children'] <0, 'children'] = df.drop(df.loc[df['children'] <0], axis=1) # удалено значение
df = df.dropna() # затем удалена вся строка
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21478.000000,21478.000000,21478.000000,21478.000000,21478.000000,21478.000000,2.147800e+04
mean,0.542276,56568.566227,43.294953,0.817208,0.972577,0.081013,1.651854e+05
std,1.381219,134935.419373,12.577312,0.548371,1.420338,0.272862,9.792208e+04
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2516.257954,33.000000,1.000000,0.000000,0.000000,1.077766e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.034062,53.000000,1.000000,1.000000,0.000000,1.956201e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


С детьми ситуация номральзована, но до сих пор присутсвуют отрицательные значение по стажу. Что будем делать с отрицательным стажем? Ввиду того, что основные задачи исследование не основываются на данных стажа работы заемщиков, было решено эти значения оставить без изменений.

# Вывод

Пропуски были заменены медианными значениями. На мой взгляд, присваивать этим заемщикам результат на основе других замещиков - не является верным. Среднеарифметическое значения использовать тоже не верно, так как большая разница в зарплате отдельных заемщиков может сильно исказить средний результат. Удалять 10 процентов наблюдений, тоже не совсем верно, они могут вносить достаточно весомый вклад в общую оценку платежеспособности. Наиболее подходщий вариант было решено оставить - медианные значения.

### Замена типа данных

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null float64
days_employed       21478 non-null float64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null float64
purpose             21478 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


Заменим дробный тип данных в графе, отражающей трудовой стаж работников и количество детей, на целочисленый, так как дни и дети не могут быть дробными.

In [81]:
df = df.astype({
    "days_employed": 'int',
    "children": 'int'
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null float64
purpose             21478 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


**Вывод**

Типы данных в столбцах days_employed и children были преобразованы в int64, так как количество дней и детей не может быть дробным.

### Обработка дубликатов

Пдсчитаем количество дубликатов, а затем удалим их и переиндексируем строки.

In [83]:
df.duplicated().sum()

54

In [84]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

Для обработки дубликатов был выбран метод drop_duplicates(), который находит дубликаты, затем .reset_index(drop=True) для нормализации индексации по порядку (переиндексации строк) с удалением старых индексов. Таким образом, дубликаты удалены. Причина их возникновения мне неизвестна. Вероятно, нужно проанализировать файл логирования внесения данных.

### Лемматизация

In [85]:
df['purpose'].head(10)

0                 покупка жилья
1       приобретение автомобиля
2                 покупка жилья
3    дополнительное образование
4               сыграть свадьбу
5                 покупка жилья
6             операции с жильем
7                   образование
8         на проведение свадьбы
9       покупка жилья для семьи
Name: purpose, dtype: object

Мы определили, что цели получения кредита указаны в свободной форме. Для автоматизирвоанного анализа, следует привести к единообразию в несколько этапов. Первый - лемматизация (им.п, ед.ч, м.р.).

In [86]:
from pymystem3 import Mystem

m = Mystem()
lemm_list = []
for purpose in df.loc[:,'purpose']:
    lemmas = m.lemmatize(purpose)
    lemm_list.append(''.join(lemmas[:-1]))
df['purpose'] = lemm_list
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьба


**Вывод**

Значения столбца 'purpose', обозначающего цель получения кредита, были лемматизированы для последующей категоризации.

Как это было сделано: 
1) был создан объект m класса Mystem();
2) был создан пустой список lemm_list для добавления лем;
3) При помощи цикла я прошелся по каждому значению в столбце 'purpose' для лемматизации (им.п, ед.ч, м.р.) значения строки и преобразовал это значение в лемму в виде спсика слов;
4) в список Lemm_list добавлялись объединенные списки при помощи метода .join() с удалением спецсимвола перевода каретки;
5) Затем временный список был записан в столбец 'purpose' нашей таблицы.

Таким образом значения столбца 'purpose' были лемматизированы и замененены.

### Категоризация данных

Определим несколько основных категорий получения кредита, путем создания словаря и присовпадении части слова - присвоим категории цифровой индекс.

In [87]:
list_of_purpose = {'недвиж': 1, 'авто': 2, 'образ': 3, 'свад': 4, 'строит': 1, 'жил': 1}

Применим словарь к нашему сету и создадим столбец с категориями займа.

In [88]:
df['purpose_category'] = 0

напишем цикл, который будет принимать ключи из словаря и сверять их со значениями столбца 11, и если слова из словаря находятся в столбце 11 - добавлять категорию в новый столбец 12

In [89]:

for row in range(len(df)):
    for purpose in list_of_purpose:
        if purpose in df.iloc[row, 11]:
            df.iloc[row, 12] = list_of_purpose[purpose]
            
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилье,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиль,2
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилье,1
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительный образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьба,4


Представлен образец обновленной таблицы с новым столбцом категорий займа.

In [90]:
df['purpose_category'].value_counts()

1    10790
2     4297
3     4004
4     2333
Name: purpose_category, dtype: int64

**Вывод**

Мы получили четыре основные категории целей получения кредита. Операцию он эту делал секунд 30..Также во время изучения таблицы, были обнаружены на первый взгляд схожие категории семейного положения. Однако объединять какие-либо из них не нахожу целесообразным, так как любые из этих состояний могут комбинироваться между собой как угодно.

## Шаг 3. Ответьте на вопросы

# - Есть ли зависимость между наличием детей и возвратом кредита в срок?

Определим, сколько наблюдейний осталось в таблице после всех преобразований, сколько всего заемщиков у нас для анализа теперь:

In [91]:
print('количество заемщиков на данный момент:',  len(df))

количество заемщиков на данный момент: 21424


Так как поставленная перед нами задача требует ответа на вопрос о взаимосвязи между наличием детей и возвращаемости кредита, и для того чтобы успешно сформировать необходимую производную таблицу, то все значения больше ноля в графе дети заменим на число один (чтобы просто обозначит факт наличия детей, который послужит индексом в производной таблице), таким образом, мы сможем подсчитать замещиков, у которых есть дети и у которых их нет.

In [92]:
df.loc[df['children'] > 0, 'children'] = 1 # где были дети проставил везде 1 чтобы были графы либо 0 либо 1, а не 3,5,20

In [93]:
no_children = df[df['children'] == 0] # выберем заемщиков без детей
with_children = df[df['children'] > 0]  # выберем заемщиков с детьми

debt_no_children = no_children[no_children['debt'] > 0] # у кого из них имеется задолженность?
debt_with_children = with_children[with_children['debt']> 0]

sum_debt_no_children = len(debt_no_children) # посчитаем количество заемщиков с задолженностью
sum_debt_with_children = len(debt_with_children)

pers_no_child = round(sum_debt_no_children / len(no_children)*100, 2) # количество должников без детей делим на людей без детей
pers_with_child = round(sum_debt_with_children / len(with_children)*100, 2) # кол-во должников без детей делим на людей без детей

children = df.pivot_table(index='children', values='debt', aggfunc=['count'])
children['people'] = sum_debt_no_children, sum_debt_with_children
children['debt in %'] = pers_no_child, pers_with_child
children

,count,people,debt in %
,debt,,
children,,,
0,14107,1063,7.54
1,7317,677,9.25


**Вывод**

Вопрос исследования: существует ли зависимость между невозвратом крелита при наличии детей, а не влияет ли коичество детей на степень невозврата. 
Вывод: Доля людей с детьми, у которых имеется задолженность на 1.71% больше тех должников, у которых нет детей, что является в пределах статистической погрешности, а значит, следует сделать вывод, что наличие детей не влияет на вероятность возврата кредита в срок.

# - Есть ли зависимость между семейным положением и возвратом кредита в срок?

Рассмотрим каждый вид семейных отношений по отдельности.

In [94]:
maried = df[df['family_status_id'] == 0] # выберем заемщиков, которые в официальном браке (статус 0)
not_official_maried = df[df['family_status_id'] == 1] # выберем заемщиков, которые в неофициальном браке (статус 1)
widow = df[df['family_status_id'] == 2] # выберем заемщиков, которые вдова/вдовец (статус 2)
divorced = df[df['family_status_id'] == 3] # выберем заемщиков, которые в разводе (статус 3)
not_married = df[df['family_status_id'] == 4] # выберем заемщиков, которые в разводе (статус 4)

debt_maried = maried[maried['debt'] > 0] # у кого из них имеется задолженность?
debt_not_official_maried = not_official_maried[not_official_maried['debt'] > 0] # у кого из них имеется задолженность?
debt_widow = widow[widow['debt'] > 0] # у кого из них имеется задолженность?
debt_divorced = divorced[divorced['debt'] > 0] # у кого из них имеется задолженность?
debt_not_married = not_married[not_married['debt'] > 0] # у кого из них имеется задолженность?

sum_debt_maried = len(debt_maried) # посчитаем количество заемщиков с задолженностью
sum_debt_not_official_maried = len(debt_not_official_maried) # посчитаем количество заемщиков с задолженностью
sum_debt_widow = len(debt_widow) # посчитаем количество заемщиков с задолженностью
sum_debt_divorced = len(debt_divorced) # посчитаем количество заемщиков с задолженностью
sum_debt_not_married = len(debt_not_married) # посчитаем количество заемщиков с задолженностью

pers_maried = round(sum_debt_maried / len(maried)*100, 2) # количество женатых должников делим на просто женатых без долгов
pers_not_official_maried = round(sum_debt_not_official_maried / len(not_official_maried)*100, 2) # кол-во должников женатых неофициально делим на людей без долгов
pers_widow = round(sum_debt_widow / len(widow)*100, 2) # количество должников без детей делим на людей без детей
pers_divorced = round(sum_debt_divorced / len(divorced)*100, 2) # кол-во должников без детей делим на людей без детей
pers_not_married = round(sum_debt_not_married / len(not_married)*100, 2) # кол-во должников без детей делим на людей без детей

family = df.pivot_table(index='family_status', values='debt', aggfunc=['count'])
family['people'] = sum_debt_not_married, sum_debt_divorced, sum_debt_widow, sum_debt_not_official_maried, sum_debt_maried  
family['debt in %'] = pers_not_married, pers_divorced,  pers_widow, pers_not_official_maried, pers_maried 
family.sort_values(by='debt in %', ascending=False)

,count,people,debt in %
,debt,,
family_status,,,
Не женат / не замужем,2805,274,9.77
гражданский брак,4158,388,9.33
женат / замужем,12315,930,7.55
в разводе,1191,85,7.14
вдовец / вдова,955,63,6.60


**Вывод**

Семейное положение оказывает крайне незначительное влияние на вероятность возвращаемого кредита. Разница между минимальным (6,60%) и максимальным (9,77%) значением, среди пяти категорий семейного положения, составляет всего 3,17%. Наибольшая вероятность невозврата среди всех пяти категорий, - у неженатых людей, хотя риск увеличивается всего на 3,17%. 

# - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим новую колонку с опредлением относительного уровня зарплаты: определим, высокая или низкая зарплата у каждого заемщика.

In [95]:
mean_salary= df['total_income'].mean() # среднеарифметическая зарплата заемщика = 165236.2753892614

def defenition_salary_level(row):
    if row['total_income'] < mean_salary:
        return "Низкая зарплата"
    if row['total_income'] > mean_salary:
        return "Высокая зарплата"


df['salary_level'] = df.apply(defenition_salary_level, axis=1)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,salary_level
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилье,1,Высокая зарплата
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиль,2,Низкая зарплата
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилье,1,Низкая зарплата
3,1,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительный образование,3,Высокая зарплата
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьба,4,Низкая зарплата


In [96]:
salary_pivot = df.pivot_table(
    index='salary_level', 
    values='debt', 
    aggfunc=['count', 'sum'])

pers_low = round(salary_pivot.loc['Низкая зарплата', 'sum'] / salary_pivot.loc['Низкая зарплата', 'count']*100, 2) # количество женатых должников делим на просто женатых без долгов
pers_high = round(salary_pivot.loc['Высокая зарплата', 'sum'] / salary_pivot.loc['Высокая зарплата', 'count']*100 , 2) # кол-во должников женатых неофициально делим на людей без долгов

salary_pivot['debt in %'] = *pers_high, *pers_low 
salary_pivot.sort_values(by='debt in %', ascending=False)
salary_pivot

,count,sum,debt in %
,debt,debt,
salary_level,,,
Высокая зарплата,7665,594,7.75
Низкая зарплата,13759,1146,8.33


**Вывод**

Доля должников с доходом меньше среднего (8.33%) всего на 0,58% больше тех должников, у кого зарплата выше среднего (7,75%). Что также не дает оснований полагать о влиянии уровня дохода на возвращаемость кредита.

# - Как разные цели кредита влияют на его возврат в срок?

In [97]:
purpose_pivot = df.pivot_table(
    index='purpose_category', 
    values='debt', 
    aggfunc=['count', 'sum'])

purpose_pivot['debts in %'] = round((purpose_pivot['sum'] / purpose_pivot['count']) * 100, 2)
purpose_pivot['цель кредита'] = ['жильё', "автомобиль", "образование", "свадьба"]
purpose_pivot.sort_values(by='debts in %', ascending=False)

,count,sum,debts in %,цель кредита
,debt,debt,,
purpose_category,,,,
2,4297,402,9.36,автомобиль
3,4004,370,9.24,образование
4,2333,186,7.97,свадьба
1,10790,782,7.25,жильё


count debt - Количество заемщиков; <br>
sum debt - Количество должников среди них.

**Вывод**

Среди четырех основных целей получения кредита, наибольшая вероятность невозврата заемных средств у заемщиков, берущих средства на покупку автомобиля (9,36%), наименьшая вероятность невозврата кредита существует при операциях с недвижимостью (7,25%). Дельта между максимальным и минимальным значением составляет всего 2,11%, что неопределяет явную взаимосвясь плохих долгов с целью кредитования, так как разница невелика.

## Шаг 4. Общий вывод

Проанализиров все показатели и поставленные задачи для исследования, мы пришли к выводу, что различные категории в рамках одного рассматриваемого вопроса не дают явных приемуществ или рисков: значения колеблются от 0,5 до 3 процентов в каждом исследуемом вопросе. Таким образом, ни одна из исследуемых гипотез не показала явной взаимосвязи невозвратности кредита с тем или иным исследуемым аспектом платежеспособности клиента. 
В связи с этим, следует разработать дополнительную систему баллов, которые бы начислялись или вычитались бы от рейтинга заемщика при совокупности всех фаторов. Так при одновременном сочетании факторов замещика: вдовца или вдову с высокой зарплатой с целью кредитования на операции недвижимостью - расценивать как высоконадежного заемщика, а неженатого заемщика с невысокой зарплатой берущего заемные средства на преобретение авто - расценивать как ненадежного замщика.
Фаткоры по отдельности осбого влияния на платежеспособность не оказывают.